In [7]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set_context('talk',font_scale=.8)

import statsmodels.api as sm

In [11]:
df = pd.read_csv('combined_dataset1.csv')
df.head()

,location_id,location_name_x,year_id,mean_ozone,median_ozone,upper_ozone,lower_ozone,mean_pm,median_pm,upper_pm,...,WHO_regions,WB_income,WB_regions,prob_death_noshock,upper_death_noshock,lower_death_noshock,prob_death_wshock,upper_death_wshock,lower_death_wshock,mean_air_pollution
0,1,Global,1990,43.958706,43.957631,44.018968,43.900055,40.815004,40.024924,55.004524,...,NaN,NaN,NaN,0.187968,0.193953,0.182240,0.189199,0.195099,0.183510,28.475955
1,1,Global,1991,44.001844,44.002259,44.057867,43.938608,NaN,NaN,NaN,...,NaN,NaN,NaN,0.187205,0.193107,0.181734,0.188913,0.194803,0.183407,22.326252
2,1,Global,1992,44.051418,44.051990,44.110324,43.990683,NaN,NaN,NaN,...,NaN,NaN,NaN,0.187683,0.193135,0.182042,0.188361,0.193783,0.182757,22.349227
3,1,Global,1993,44.354340,44.354440,44.414131,44.296156,NaN,NaN,NaN,...,NaN,NaN,NaN,0.188739,0.194052,0.183662,0.189509,0.194808,0.184446,22.498218
4,1,Global,1994,44.570022,44.569441,44.628212,44.511082,NaN,NaN,NaN,...,NaN,NaN,NaN,0.189720,0.195105,0.184711,0.193410,0.198740,0.187799,22.603008


In [12]:
df_new = df[['location_id', 'location_name_x', 'mean_air_pollution', 'mean_ozone', 'mean_pm', 'mean_hap', 'sdi_quintile', 'WB_income', 'WB_regions']]
df_new

,location_id,location_name_x,mean_air_pollution,mean_ozone,mean_pm,mean_hap,sdi_quintile,WB_income,WB_regions
0,1,Global,28.475955,43.958706,40.815004,0.654155,NaN,NaN,NaN
1,1,Global,22.326252,44.001844,NaN,0.650659,NaN,NaN,NaN
2,1,Global,22.349227,44.051418,NaN,0.647037,NaN,NaN,NaN
3,1,Global,22.498218,44.354340,NaN,0.642097,NaN,NaN,NaN
4,1,Global,22.603008,44.570022,NaN,0.635995,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
20663,44792,Plymouth,14.616652,33.777512,10.072050,0.000393,High SDI,World Bank High Income,Europe & Central Asia - WB
20664,44792,Plymouth,14.301855,33.167273,9.737908,0.000383,High SDI,World Bank High Income,Europe & Central Asia - WB
20665,44792,Plymouth,14.194551,32.929674,9.653606,0.000373,High SDI,World Bank High Income,Europe & Central Asia - WB
20666,44792,Plymouth,14.676422,34.307616,9.721287,0.000363,High SDI,World Bank High Income,Europe & Central Asia - WB


In [13]:
cat_cols = [x for x in df_new.columns.to_list() if x in ['sdi_quintile','WB_income','WB_regions']]
df_dummy = df_new
for var in cat_cols:
    dummy = pd.get_dummies(df_dummy[var], prefix = var)
    df_dummy = pd.concat([df_dummy, dummy], axis = 1)
    df_dummy = df_dummy.drop(columns = var)

In [15]:
df_dummy.columns

Index(['location_id', 'location_name_x', 'mean_air_pollution', 'mean_ozone',
       'mean_pm', 'mean_hap', 'sdi_quintile_High Mid SDI',
       'sdi_quintile_High SDI', 'sdi_quintile_Low Mid SDI',
       'sdi_quintile_Low SDI', 'sdi_quintile_Mid SDI',
       'WB_income_World Bank High Income', 'WB_income_World Bank Low Income',
       'WB_income_World Bank Lower Middle Income',
       'WB_income_World Bank Upper Middle Income',
       'WB_regions_East Asia & Pacific - WB',
       'WB_regions_Europe & Central Asia - WB',
       'WB_regions_Latin America & Caribbean - WB',
       'WB_regions_Middle East & North Africa - WB',
       'WB_regions_North America', 'WB_regions_South Asia - WB',
       'WB_regions_Sub-Saharan Africa - WB'],
      dtype='object')

## OLS

In [21]:
y = df_dummy.mean_air_pollution
#X = df_dummy[['sdi_quintile_High Mid SDI', 'sdi_quintile_High SDI', 'sdi_quintile_Low Mid SDI', 'sdi_quintile_Low SDI', 'sdi_quintile_Mid SDI']]

#X = df_dummy[['WB_income_World Bank High Income', 'WB_income_World Bank Low Income', 'WB_income_World Bank Lower Middle Income', 'WB_income_World Bank Upper Middle Income']]

#X = df_dummy[['WB_regions_East Asia & Pacific - WB', 'WB_regions_Europe & Central Asia - WB', 'WB_regions_Latin America & Caribbean - WB', 'WB_regions_Middle East & North Africa - WB', 'WB_regions_North America', 'WB_regions_South Asia - WB', 'WB_regions_Sub-Saharan Africa - WB']]
# all
X = df_dummy[['sdi_quintile_High Mid SDI', 'sdi_quintile_High SDI', 'sdi_quintile_Low Mid SDI', 'sdi_quintile_Low SDI', 'sdi_quintile_Mid SDI', 'WB_income_World Bank High Income', 'WB_income_World Bank Low Income', 'WB_income_World Bank Lower Middle Income', 'WB_income_World Bank Upper Middle Income', 'WB_regions_East Asia & Pacific - WB', 'WB_regions_Europe & Central Asia - WB', 'WB_regions_Latin America & Caribbean - WB', 'WB_regions_Middle East & North Africa - WB', 'WB_regions_North America', 'WB_regions_South Asia - WB', 'WB_regions_Sub-Saharan Africa - WB']]

X = sm.add_constant(X)

In [22]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     mean_air_pollution   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.233
Method:                 Least Squares   F-statistic:                     393.3
Date:                Thu, 18 Nov 2021   Prob (F-statistic):               0.00
Time:                        13:48:12   Log-Likelihood:                -62294.
No. Observations:               20668   AIC:                         1.246e+05
Df Residuals:                   20651   BIC:                         1.248e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------